## Сброс забытого root пароля MySQL

    mysql -u root -p                                                           
    Enter password:                                                                 
    ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES) 

Остановить сервер  

    sudo service mysqld stop

В файле /etc/mysql/mysql.conf.d/mysqld.cnf

    [mysqld]
    # отключение проверки пароля
    skip-grant-tables

Запустить сервер  

    sudo service mysqld start

В консоли mysql:

    $ mysql -u root
    mysql> use mysql;
    mysql> show tables;
    mysql> describe user;
    mysql> SELECT authentication_string FROM mysql.user;
    ... # хеш пароля (ничего не дает)
    
    mysql> UPDATE mysql.user SET authentication_string=PASSWORD('pass') WHERE user='root';
    
  mysql> UPDATE mysql.user SET authentication_string=CONCAT('*', UPPER(SHA1(UNHEX(SHA1('pass'))))) WHERE user='root';
    
    mysql> FLUSH PRIVILEGES;
    mysql> exit

Убрать строку в файле /etc/mysql/mysql.conf.d/mysqld.cnf. Настройки подхвятятся автоматически.

## Аутентификация пользователя в MySQL без пароля (по сокету)

В файле /etc/mysql/mysql.conf.d/mysqld.cnf

    [mysqld]
    ...
    # плагин аутентификации по имени польтзователя через сокет
    plugin-load-add=auth_socket.so

Либо в консоли 

    INSTALL PLUGIN auth_socket SONAME 'auth_socket.so';

В консоли

    $ mysql -u root -p
    Enter password: 
    mysql> CREATE USER 'user1'@'localhost' IDENTIFIED WITH auth_socket;                                                                                                           
    mysql> GRANT ALL PRIVILEGES ON *.* TO 'user1'@'localhost';                                                                                                          
    mysql> SELECT plugin from mysql.user where User='user1';                                                                                  
    ...
    auth_socket
    
    $ mysql

https://github.com/IsFilimonov/LearningPath/

Для работы с базой лично я использую IDE DataGrip (платная) или DBeaver (бесплатная) - под MacOS.  
DBeaver - все возможные виды баз данных.  
Чисто для MySQL - MySQL Workbench.  

Это всё сторонние решения, но можно воспользоваться более простым способом - использовать Jupyter Notebook в роли SQL IDE. Для этого нам потребуется установить несколько библиотек. Для работы с Jupyter Notebook я использую платформу Anaconda, поэтому все нужные библиотеки для работы я буду устанавливать именно в эту среду. Я не буду останавливаться на деталях в вопросе "как?", чтобы в деталях не закопаться. В интернете полно подсказок. Главное, должны быть установлены:

    ipython-sql - пакет для работы с SQL запросами;
    psycopg2 или psycopg2-binary - драйвер базы данных Postgres. Там есть нюансы с установкой;
    mysql - драйвер MySQL;
    sqlalchemy - ORM для работы с SQL БД.
    
    (py38) ➜  projects pip install mysql ipython-sql psycopg2 sqlalchemy

In [1]:
import sqlalchemy
sqlalchemy.create_engine("mysql://localhost:3306/mysql")

Engine(mysql://localhost:3306/mysql)

In [2]:
%load_ext sql

In [3]:
%sql mysql://localhost:3306/mysql

In [5]:
%%sql

SHOW DATABASES;

 * mysql://localhost:3306/mysql
14 rows affected.


Database
billing_simple
information_schema
module24
module31
module33
module35
module41
mysql
performance_schema
project_simple


## PostgreSQL

    sudo apt install postgresql

Будет создан пользователь по-умолчанию postgres

    sudo -i -u postgres

    ➜  ~  sudo -i -u postgres
    postgres@Z87N:~$ psql
    psql (12.8 (Ubuntu 12.8-0ubuntu0.20.04.1))
    Type "help" for help.

    postgres=# \conninfo
    You are connected to database "postgres" as user "postgres" via socket in "/var/run/postgresql" at port "5432".


Для админа (portgres) авторизация по сокету включена по умолчанию. Имеет смысл создать одноименного пользователя.

    postgres@Z87N:~$ createuser --interactive                                                                                      
    Enter name of role to add: user1
    Shall the new role be a superuser? (y/n) y

    createdb user1
    
    $ psql

Все, локальный пользователь user1 логинится без пароля.  
Остальные настройки /etc/postgresql/12/main/pg_hba.conf.  
По-умолчанию все сетевые подключения по паролю md5.

In [52]:
# чтобы sqlalchemy идентифицировалась по пользователю, обязательно убрать localhost
import sqlalchemy
sqlalchemy.create_engine("postgresql:///postgres")

Engine(postgresql:///postgres)

In [53]:
%reload_ext sql

In [54]:
%sql postgresql:///postgres

In [57]:
%%sql

SELECT version();

 * postgresql:///postgres
1 rows affected.


version
"PostgreSQL 12.8 (Ubuntu 12.8-0ubuntu0.20.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit"
